In [ ]:
%pip install faker

In [ ]:
%pip install pandas

In [ ]:
import faker
fake = faker.Faker()

In [ ]:
user_profile = {
    "user_id": fake.random_int(min=1, max=999),
    "name": fake.name(),
    "age": fake.random_int(min=18, max=75),
    "city": fake.city(),
    "fav_category": fake.random_element(elements=("Drama", "Action", "Adventure", "Comedy", "Sci-Fi"))
}
user_profile

In [ ]:
user_profile = {
    "user_id": fake.random_int(min=1, max=999),
    "name": fake.name(),
    "age": fake.random_int(min=18, max=75),
    "fav_category": fake.random_element(elements=("Drama", "Action", "Adventure", "Comedy", "Sci-Fi",
        'Kids Accessories',
        'Kids Clothing',
        'Kids Shoes',
        'Men Accessories',
        'Men Clothing',
        'Men Shoes',
        'Running Accessories',
        'Running Shoes',
        'Soccer Accessories',
        'Soccer Shoes',
        'Women Accessories',
        'Women Clothing',
        'Women Shoes',))
        }
user_profile

In [ ]:
# Download IMDB data
data_dir = '../../assets'
imdb_dir = data_dir
!wget -P $imdb_dir https://d2peeor3oplhc6.cloudfront.net/personalize-immersionday-media/imdb/items.csv

In [ ]:
# read sample data
import pandas as pd
assets_dir = '../../assets'
movie_data = pd.read_csv(assets_dir + '/movie_items.csv', sep=',', dtype={'PROMOTION': "string"})
retail_data = pd.read_csv(assets_dir + '/adidas_items.csv', sep=',', dtype={'PROMOTION': "string"})
travel_data = pd.read_csv(assets_dir + '/travel_items.csv', sep=',', dtype={'PROMOTION': "string"})
# retail_data.head(5)
# travel_data.head(5)
# return recommended items and return name and description
# print only name and description from item_data
movie_data.head(5)


In [ ]:
def get_top_items(type, category, n=3):
    if type == 'movie':
        data = movie_data[movie_data['GENRES'].str.contains(category)].sort_values(by='IMDB_RATING', ascending=False).head(3)
        data = data.rename(columns={'TITLE': 'name'})
        data = data.rename(columns={'PLOT': 'description'})
        return data[['TITLE', 'PLOT']]
    elif type == 'retail':
        data = retail_data[retail_data['breadcrumbs'].str.contains(category)].sort_values(by='average_rating', ascending=False).head(3)
        return data[['name', 'description']]
    elif type == 'travel':
        data = travel_data[travel_data['DST_CITY'].str.contains(category)].sort_values(by='NUMBER_OF_SEARCH_BY_USER', ascending=False).head(3)
        data['description'] = data.apply(lambda row: 'Travel from ' + str(row['SRC_CITY']) + ' on ' + str(row['AIRLINE'] + ' for a ' + str(row['DURATION_DAYS']) + ' trip in ' + str(row['MONTH']) + ' - $' + str(row['DYNAMIC_PRICE'])), axis=1)
        data = data.rename(columns={'DST_CITY': 'name'})
        return data[['name', 'description']]

In [ ]:
get_top_items("travel", "Hong Kong")

In [ ]:
# download the s3 file from "s3://personalize-solution-staging-us-east-1/personalize-immersionday-travel/travel_items.csv"
import boto3
import pandas as pd
s3 = boto3.client('s3')
def download_s3_file(bucket, key, filename):
    s3.download_file(bucket, key, filename)
    df = pd.read_csv(filename)
    return df
df = download_s3_file('personalize-solution-staging-us-east-1', 'personalize-immersionday-travel/travel_items.csv', 'travel_items.csv')
print(df.head())

In [ ]:
get_top_items("movie","Action")

In [ ]:
items = ["Drama", "Action", "Adventure", "Comedy", "Sci-Fi",
        'Kids Accessories', 'Kids Clothing', 'Kids Shoes', 'Men Accessories', 'Men Clothing',
        'Men Shoes', 'Running Accessories', 'Running Shoes', 'Soccer Accessories', 'Soccer Shoes',
        'Women Accessories', 'Women Clothing', 'Women Shoes', 
        'Beijing', 'Guangzhou','Hong Kong','London', 'New York', 'Seoul', 'Shanghai', 'Sydney', 'Tokyo']

[{"domain": "travel", "category": item} for item in items]

In [ ]:
[{'domain': 'movie', 'category': 'Drama'},
 {'domain': 'movie', 'category': 'Action'},
 {'domain': 'movie', 'category': 'Adventure'},
 {'domain': 'movie', 'category': 'Comedy'},
 {'domain': 'movie', 'category': 'Sci-Fi'},
 {'domain': 'retail', 'category': 'Kids Accessories'},
 {'domain': 'retail', 'category': 'Kids Clothing'},
 {'domain': 'retail', 'category': 'Kids Shoes'},
 {'domain': 'retail', 'category': 'Men Accessories'},
 {'domain': 'retail', 'category': 'Men Clothing'},
 {'domain': 'retail', 'category': 'Men Shoes'},
 {'domain': 'retail', 'category': 'Running Accessories'},
 {'domain': 'retail', 'category': 'Running Shoes'},
 {'domain': 'retail', 'category': 'Soccer Accessories'},
 {'domain': 'retail', 'category': 'Soccer Shoes'},
 {'domain': 'retail', 'category': 'Women Accessories'},
 {'domain': 'retail', 'category': 'Women Clothing'},
 {'domain': 'retail', 'category': 'Women Shoes'},
 {'domain': 'travel', 'category': 'Beijing'},
 {'domain': 'travel', 'category': 'Guangzhou'},
 {'domain': 'travel', 'category': 'Hong Kong'},
 {'domain': 'travel', 'category': 'London'},
 {'domain': 'travel', 'category': 'New York'},
 {'domain': 'travel', 'category': 'Seoul'},
 {'domain': 'travel', 'category': 'Shanghai'},
 {'domain': 'travel', 'category': 'Sydney'},
 {'domain': 'travel', 'category': 'Tokyo'}]

In [ ]:
import os
from PIL import Image
original_dir = '../../assets/stock_originals'
thumbs_dir = '../../assets/stock_thumbs'
size = 128, 128  # Desired thumbnail size

# Loop through all JPEG files in the current directory
for infile in os.listdir(original_dir):
    print(infile)
    if infile.endswith(".jpeg"):
        im = Image.open(os.path.join(original_dir,infile))
        im.thumbnail(size)
        im.save(f"{thumbs_dir}/thumbnail_{os.path.splitext(infile)[0]}.jpeg", "JPEG")

In [ ]:
%pip install sagemaker

In [ ]:
%pip list


In [ ]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel
boto_session = boto3.Session(profile_name='amtdemo-cli')
sagemaker_session = sagemaker.Session(boto_session=boto_session)
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto_session.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']
	print(role)

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'Salesforce/blip2-flan-t5-xl',
	'HF_TASK':'image-to-text'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.37.0',
	pytorch_version='2.1.0',
	py_version='py310',
	env=hub,
	role=role,
	sagemaker_session=sagemaker_session
)
end_point_name = 'blip2-flan-t5-xl-test'
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.g5.2xlarge', # ec2 instance type
	endpoint_name=end_point_name
)
print(predictor.endpoint_name)

In [ ]:
# setup clients
import sagemaker
import boto3
boto_session = boto3.Session(profile_name='amtdemo-cli')
sagemaker_session = sagemaker.Session(boto_session=boto_session)

# Create a SageMaker runtime client
runtime = boto_session.client('sagemaker-runtime')
# endpoint_name = predictor.endpoint_name
endpoint_name = "huggingface-pytorch-inference-2024-08-21-14-51-46-820"

In [ ]:
# get the image data ready
from PIL import Image
import base64
import io
original_dir = '../../assets/stock_originals'
# Open the image
fname = "AdobeStock_26803377.jpeg"
image = Image.open(f"{original_dir}/{fname}")
rimage = image.resize((1024, 640))
# Convert the image to a base64 string
buffered = io.BytesIO()
rimage.save(f"{original_dir}/Resized_{fname}", format="JPEG")
image.save(buffered, format="JPEG")
byte_array = buffered.getvalue()
img_str = base64.b64encode(byte_array).decode("utf-8")


In [ ]:
# Send the request to the BLIP-2 model
import json
prompt = """Question: Describe the image in detailed 20 words? Answer:"""
# inputs = {"inputs": {"image":img_str, "prompt":prompt}}
inputs = {"inputs": img_str, "parameters": {"prompt":prompt}}
# print(json.dumps(inputs, indent=2))
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    # EndpointName="blip-2-test",
    ContentType="application/json",
    # Body=bytearray(json.dumps({"inputs": img_str, "text":prompt}), "utf-8"),
    Body=bytearray(json.dumps(inputs), "utf-8")
)

# Decode the response
result = json.loads(response["Body"].read().decode("utf-8"))
# Print the generated caption
print(result[0]["generated_text"])

In [ ]:
neg_prompt = "worst quality, low quality, normal quality, lowres, low details, over saturated, under saturated, overexposed, underexposed, grayscale, b&w, bad photo, bad photography, bad art:1.4), (watermark, signature, text font, username, error, logo, words, letters, digits, autograph, trademark, name:1.2), (blur, blurry, grainy),ugly, asymmetrical, mutated malformed, mutilated, poorly lit, bad shadow, cropped, out of frame, cut off, out of focus, glitch"

In [ ]:
def get_supported_img_size(inp_width, aspect_ratio):
    aspect_ratios = (1,0.67,1.5,0.6,1.67,0.78,1.28,0.54,1.83,0.45,2.2,1.83)
    sizes = [
        (1024,1),
        (768,1),
        (512,1),
        (768,0.67),
        (384,0.67),
        (1152,1.5),
        (576,1.5),
        (768,0.6),
        (384,0.6),
        (1280,1.67),
        (640,1.67),
        (896,0.78),
        (448,0.78),
        (1152,1.28),
        (576,1.28),
        (768,0.54),
        (384,0.54),
        (1408,1.83),
        (704,1.83),
        (640,0.45),
        (320,0.45),
        (1408,2.2),
        (704,2.2),
        (1152,1.8),
        (1173,1.83),
    ]
    # for the given aspect ratio find the closest aspect ratio aspect_ratios
    closest_aspect_ratio = min(aspect_ratios, key=lambda x: abs(x - aspect_ratio))
    # for the closest_aspect_ratio and input_width find the best tuple
    closest_match = min(sizes, key=lambda x: abs(x[0] - inp_width) + abs(x[1] - closest_aspect_ratio))
    return closest_match[0], closest_match[1]

In [ ]:
get_supported_img_size(1173704, 3)

In [ ]:
import boto3
from botocore.exceptions import ClientError
import logging
logger = logging.getLogger(__name__)

profile="genai"

def get_available_models(profile):
    session = boto3.Session(profile_name=profile)
    client = session.client('bedrock')
    try:
        models = client.list_foundation_models()
    except ClientError as e:
        logger.error(e.response)
        return None
    return models["modelSummaries"]

def get_filtered_models(profile, output_mode='IMAGE'):
    models = get_available_models(profile)
    filtered_models = []
    for model in models:
        if output_mode in model["outputModalities"]:
            filtered_models.append(model["modelId"])
    return filtered_models

get_filtered_models(profile)